In [2]:
# 0314 v2 - 가까운 두 개 마커로 위치 파악
# 마커 개수 늘림 시야에 3개까지 잘 잡히는데 가까운 2개를 고르려다 보니까 어떤 마커 2개를 고르는지에 따라서 좌표가 다르게 나타남
#v3: 여러개 중에 2개씩 이용해서 위치 뽑고 평균 내기
#v4: 칼만 필터  - v3보다 개선됨 (튀는 값이 줄어들었지만 여전히 많긴 함)

# 0317 v1: z축 거리 고정해서 입력하기 + 칼만필터까지 적용
# 정확도 높음 z값 -1.8m로 했는데 로봇에 부착하고 다시 거리 재보면 될듯
# v2: 마커 17번 빠졌던 거 추가
# v3: 마커 z축 회전 정렬 
#v4: 회전 정렬 결과 나타나게 opencv 3d 축 그리기 반영
# v5: 로봇 제어를 위한 yaw 행렬 추출  -- 아직 안됨 아래 코드 실행해보고 수정할 것 


# 0505 v1: 로봇 제어를 위한 yaw 행렬 추출 부터 ( 단, Euler 변환까지 적용)

## 성공 - 마커 90도씩 돌려보면 결과 잘 나옴 (천장 마커로는 아직 테스트 못함

# 0516 : 추출한 행렬가지고 로봇 제어 시도
# yaw 추출 
# 다중 waypoint 설정해서 움직임 제어

In [5]:
import cv2
import cv2.aruco as aruco
import numpy as np
from scipy.spatial.transform import Rotation as R

# ===========================
# 📌 1. 설정 및 초기값
# ===========================
cameraMatrix = np.load('C:/cal_cam/calibration_matrix.npy')
distCoeffs = np.load('C:/cal_cam/distortion_coefficients.npy')
marker_length = 0.16  # 마커 실제 길이 (m)
FIXED_Z = -1.8  # 고정된 Z 값 (m)

# 마커 월드 좌표
marker_world_pos = {
    0: np.array([0.0, 0.0, 0.0]), 1: np.array([1.5, 0.0, 0.0]),
    2: np.array([3.0, 0.0, 0.0]), 3: np.array([4.5, 0.0, 0.0]),
    4: np.array([0.0, 1.5, 0.0]), 5: np.array([1.5, 1.5, 0.0]),
    6: np.array([3.0, 1.5, 0.0]), 7: np.array([4.5, 1.5, 0.0]),
    8: np.array([0.0, 3.0, 0.0]), 9: np.array([1.5, 3.0, 0.0]),
    10: np.array([3.0, 3.0, 0.0]), 11: np.array([4.5, 3.0, 0.0]),
    12: np.array([0.7, 0.7, 0.0]), 13: np.array([2.3, 0.7, 0.0]),
    14: np.array([3.7, 0.7, 0.0]), 15: np.array([0.7, 2.3, 0.0]),
    16: np.array([2.3, 2.3, 0.0]), 17: np.array([3.7, 2.3, 0.0])
}

# 마커별 z축 회전 보정 (각도)
marker_orientation = {
    0: 0, 1: 0, 2: -180, 3: 0,
    4: -180, 5: -180, 6: 0, 7: -180,
    8: 0, 9: 0, 10: -180, 11: 0,
    12: -180, 13: -90, 14: -270,
    15: 0, 16: -90, 17: -90
}

# 칼만 필터 초기화 함수
def init_kalman_filter():
    kf = cv2.KalmanFilter(6, 3)
    kf.measurementMatrix = np.eye(3, 6, dtype=np.float32)
    kf.transitionMatrix = np.array([
        [1, 0, 0, 1, 0, 0],
        [0, 1, 0, 0, 1, 0],
        [0, 0, 1, 0, 0, 1],
        [0, 0, 0, 1, 0, 0],
        [0, 0, 0, 0, 1, 0],
        [0, 0, 0, 0, 0, 1],
    ], dtype=np.float32)
    kf.processNoiseCov = np.eye(6, dtype=np.float32) * 1e-4
    kf.measurementNoiseCov = np.eye(3, dtype=np.float32) * 1e-2
    return kf

kalman_filters = {}

# z축 회전 보정 행렬 생성 함수
def get_z_rotation_matrix(degrees):
    return R.from_euler('z', degrees, degrees=True).as_matrix()

# ===========================
# 📌 2. 카메라 열기 및 메인 루프
# ===========================
cap = cv2.VideoCapture(0) #####################
cap.set(cv2.CAP_PROP_FRAME_WIDTH, 1280)
cap.set(cv2.CAP_PROP_FRAME_HEIGHT, 720)
cap.set(cv2.CAP_PROP_EXPOSURE, -4)

aruco_dict = aruco.getPredefinedDictionary(aruco.DICT_4X4_50)
detector_params = aruco.DetectorParameters()

if not cap.isOpened():
    print("카메라 열기 실패!")
    exit()

while True:
    ret, frame = cap.read()
    if not ret:
        print("프레임 읽기 실패!")
        break

    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    corners, ids, _ = aruco.detectMarkers(gray, aruco_dict, parameters=detector_params)

    if ids is not None:
        aruco.drawDetectedMarkers(frame, corners, ids)
        rvecs, tvecs, _ = aruco.estimatePoseSingleMarkers(corners, marker_length, cameraMatrix, distCoeffs)

        cam_positions = []
        yaws = []
        weights = []

        for i, m_id in enumerate(ids.flatten()):
            if m_id in marker_world_pos:
                rvec, tvec = rvecs[i], tvecs[i]
                world_pos = marker_world_pos[m_id]

                # ✅ 보정 회전
                R_marker, _ = cv2.Rodrigues(rvec)
                correction_R = get_z_rotation_matrix(marker_orientation.get(m_id, 0))
                R_corrected = correction_R @ R_marker
                rvec_corrected, _ = cv2.Rodrigues(R_corrected)  # 보정된 회전 벡터

                # ✅ 3D 축 그리기
                cv2.drawFrameAxes(frame, cameraMatrix, distCoeffs, rvec_corrected, tvec, 0.1)

                # ✅ Yaw 추출
                euler_angles = R.from_matrix(R_corrected).as_euler('xyz', degrees=True)
                yaw = euler_angles[2]  # Z축 회전
                yaws.append(yaw)

                # ✅ 카메라 위치
                cam_pos = -R_marker.T @ tvec.reshape(3, 1)
                cam_pos[2] = FIXED_Z  # Z 고정
                cam_pos = (correction_R @ cam_pos + world_pos.reshape(3, 1)).flatten()

                # 칼만 필터
                if m_id not in kalman_filters:
                    kalman_filters[m_id] = init_kalman_filter()
                kf = kalman_filters[m_id]
                kf.correct(cam_pos.astype(np.float32))  # 관측
                estimate = kf.predict()[:3]
                cam_positions.append(estimate)

                # 거리 기반 가중치
                distance = np.linalg.norm(tvec)
                weights.append(1.0 / (distance + 1e-6))

        # ✅ 가중 평균 위치 및 방향
        weights = np.array(weights)
        weights /= np.sum(weights)
        camera_world_pos = np.average(cam_positions, axis=0, weights=weights)
        camera_yaw = np.average(yaws, weights=weights)

        # ✅ 출력

        cx, cy, cz = camera_world_pos

        cx = float(cx)
        cy = float(cy)
        cz = float(cz)
        camera_yaw = float(camera_yaw)

        
        #cx, cy, cz = camera_world_pos
        print(f"[INFO] Position: ({cx:.2f}, {cy:.2f}, {cz:.2f}), Yaw: {camera_yaw:.2f}°")
        cv2.putText(frame, f"Pos: X={cx:.2f} Y={cy:.2f} Z={cz:.2f} Yaw={camera_yaw:.2f}deg",
                    (30, 40), cv2.FONT_HERSHEY_SIMPLEX, 0.6, (0, 0, 255), 2)

    cv2.imshow("3D Axis with Yaw", frame)
    if cv2.waitKey(1) & 0xFF == 27:
        break

cap.release()
cv2.destroyAllWindows()


C:\Users\user\AppData\Local\Temp\ipykernel_8004\725015293.py:136: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cx = float(cx)
C:\Users\user\AppData\Local\Temp\ipykernel_8004\725015293.py:137: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cy = float(cy)
C:\Users\user\AppData\Local\Temp\ipykernel_8004\725015293.py:138: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  cz = float(cz)


[INFO] Position: (0.00, 0.00, 0.00), Yaw: -1.56°
[INFO] Position: (0.03, 0.00, -0.02), Yaw: -1.56°
[INFO] Position: (0.14, -0.00, -0.09), Yaw: -0.77°
[INFO] Position: (0.45, -0.00, -0.28), Yaw: 0.21°
[INFO] Position: (1.04, -0.01, -0.63), Yaw: 0.21°
[INFO] Position: (1.80, -0.02, -1.11), Yaw: 2.66°
[INFO] Position: (2.56, -0.02, -1.57), Yaw: 2.66°
[INFO] Position: (3.14, -0.02, -1.93), Yaw: 3.27°
[INFO] Position: (3.50, -0.02, -2.16), Yaw: 3.27°
[INFO] Position: (3.69, -0.02, -2.28), Yaw: 3.86°
[INFO] Position: (3.74, -0.02, -2.31), Yaw: 3.86°
[INFO] Position: (3.72, -0.01, -2.30), Yaw: 3.42°
[INFO] Position: (3.64, -0.01, -2.25), Yaw: 3.42°
[INFO] Position: (3.32, -0.01, -2.04), Yaw: -3.68°
[INFO] Position: (3.21, -0.00, -1.97), Yaw: -3.68°
[INFO] Position: (3.33, 0.00, -2.04), Yaw: 2.69°
[INFO] Position: (3.23, 0.01, -1.97), Yaw: 2.12°
[INFO] Position: (3.15, 0.02, -1.92), Yaw: 2.12°
[INFO] Position: (3.09, 0.02, -1.88), Yaw: 1.78°
[INFO] Position: (3.03, 0.02, -1.84), Yaw: 1.78°
[IN